In [ ]:
import networkx as nx
import pandas as pd
import numpy as np

In [ ]:
"""
G = nx.DiGraph()
edges = [(0, 1, 2.5), (1, 2, 1.5), (2, 0, 3.0), (2, 3, 2.0), (3, 4, 1.0)]
G.add_weighted_edges_from(edges)
"""

In [ ]:
def get_graph_from_network(trade_data):
    """
    Analyzes trade network using Infomap community detection.
    
    Parameters:
    trade_data (pd.DataFrame): DataFrame with columns 'reporterDesc', 'partnerDesc', and 'primaryValue'
    
    Returns:
    tuple: (node_communities, node_mapping)
    """
    # Create a graph
    G = nx.DiGraph()

    # Add edges to the graph, excluding 'World'
    for index, row in trade_data.iterrows():
        source = row['reporterDesc']
        dest = row['partnerDesc']
        value = row['primaryValue']
        if source != 'World' and dest != 'World':
            if G.has_edge(source, dest):
                G[source][dest]['weight'] += value
            else:
                G.add_edge(source, dest, weight=value)
    # # Normalize the weights of the edges
    # for u in G.nodes():
    #     total_weight = sum(data['weight'] for _, _, data in G.edges(u, data=True))
    #     for v in G.successors(u):
    #         G[u][v]['weight'] /= total_weight
    # Create a mapping between string names and numeric IDs
    nodes = list(G.nodes())
    node_to_id = {node: idx for idx, node in enumerate(nodes, start=1)}
    id_to_node = {idx: node for node, idx in node_to_id.items()}

    # Print network statistics
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    
    # # Print the edges with weights
    # print("\nEdges with weights:")
    # for u, v, data in G.edges(data=True):
    #     print(f"({u}, {v}, {data})")

    

    return G, node_to_id,id_to_node

# Example usage:

# Read the trade data
trade_data_2024 = pd.read_excel('TradeData.xlsx')

# Analyze the network
G, node_to_id, id_to_node = get_graph_from_network(trade_data_2024)
centrality = nx.betweenness_centrality(G)
centrality_keys = list(reversed(sorted(centrality.keys(),key = lambda x: centrality[x])))
print([(key,centrality[key]) for key in centrality_keys[:5]])
print(len(nx.community.louvain_communities(G)))

In [ ]:
def create_weighted_directed_graph(edges):
    G = nx.DiGraph()  # Create a directed graph
    G.add_weighted_edges_from(edges)  # Add edges with weights
    return G

## Betweenness Centrality

In [13]:
def calculate_betweenness_centrality(G):
    # Calculate betweenness centrality with weights
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight', normalized=True)
    return betweenness_centrality

## Out degree Centrality 

In [ ]:
import networkx as nx

def calculate_out_degree_centrality(G):
    """
    Calculates the out-degree centrality for each node in a directed weighted graph,
    ignoring the weights on the edges.
    :param G: A directed weighted graph (NetworkX DiGraph).
    :return: Dictionary with nodes as keys and their out-degree centrality as values.
    """
    N = len(G.nodes)  # Total number of nodes
    centrality = {}

    for node in G.nodes:
        # Calculate the out-degree (number of outgoing edges, without considering weights)
        out_degree = G.out_degree(node)
        
        # Calculate out-degree centrality
        centrality[node] = out_degree / (N - 1) if N > 1 else 0  # Avoid division by zero for single-node graph

    return centrality

Out-degree centrality for each node:
Node 0: 0.5000
Node 1: 1.0000
Node 2: 0.0000


### Radiality centrality :- https://search.r-project.org/CRAN/refmans/centiserve/html/radiality.html

## Eccentricity and Closeness and Radiality

In [ ]:
import networkx as nx

def calculate_metrics(G):
    """
    Calculates radiality, eccentricity, and closeness centrality for each node in a directed weighted graph.
    :param G: A directed weighted graph (NetworkX DiGraph).
    :return: Three dictionaries with nodes as keys for radiality, eccentricity, and closeness centrality.
    """
    N = len(G.nodes)  # Total number of nodes
    radiality = {}
    eccentricity = {}
    closeness_centrality = {}

    # Step 1: Calculate all pairs shortest paths in the directed weighted graph using Dijkstra
    shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight="weight"))
    
    # Step 2: Calculate the graph diameter as the longest shortest path distance
    # Note: This considers only reachable pairs to avoid infinity issues
    diameter = max(
        max(lengths.values()) for lengths in shortest_paths.values() if lengths
    )

    for v in G.nodes:
        # Initialize variables for radiality, eccentricity, and closeness centrality for node v
        sum_adjusted_paths = 0
        total_distance = 0
        max_distance = 0

        # Step 3: Calculate metrics based on shortest path distances
        for u in G.nodes:
            if u != v:
                # Get the shortest path distance from v to u
                distance_vu = shortest_paths[v].get(u, float('inf'))

                # Radiality: Sum (G + 1 - distance(v, u)) for each reachable node u
                if distance_vu < float('inf'):
                    adjusted_distance = (diameter + 1 - distance_vu)
                    sum_adjusted_paths += adjusted_distance
                    total_distance += distance_vu
                    max_distance = max(max_distance, distance_vu)

        # Radiality for node v
        radiality[v] = sum_adjusted_paths / (N - 1) if N > 1 else 0

        # Eccentricity for node v (maximum distance to any reachable node)
        eccentricity[v] = max_distance if max_distance > 0 else float('inf')

        # Closeness centrality for node v (inverse of average shortest path distance)
        closeness_centrality[v] = 1 / total_distance if total_distance > 0 else 0

    return radiality, eccentricity, closeness_centrality


## Now we need to implement the Trade_weight matrix, find the bottleneck node

### Finding the bottleneck is graph before news comes

In [ ]:
import numpy as np

def create_loss_weight_adj_matrix(G):
    L = nx.to_numpy_array(G, weight="weight")
    
    # Transpose the matrix to make each column represent imports for normalization
    L_T = L.T
    
    # Normalize each row of the transposed matrix to get import-based percentages
    for i in range(L_T.shape[0]):
        col_sum = np.sum(L_T[i])
        if col_sum > 0:  # Avoid division by zero for nodes with no imports
            L_T[i] = L_T[i] / col_sum
    
    # Transpose back to the original orientation
    L_normalized = L_T.T
    return L_normalized


# Metrics on trade-weighted graph
radiality, eccentricity, closeness_centrality = calculate_metrics(G)
betweenness_centrality = calculate_betweenness_centrality(G)

# Metrics on loss-weighted graph
# Create a directed weighted graph using the loss-weighted adjacency matrix
L = create_loss_weight_adj_matrix(G)
G_loss = nx.from_numpy_array(L, create_using=nx.DiGraph)
radiality_loss, eccentricity_loss, closeness_centrality_loss = calculate_metrics(G_loss)
betweenness_centrality_loss = calculate_betweenness_centrality(G_loss)

# Function to get top 5 nodes based on metric values
def get_top_5(metric_dict):
    return sorted(metric_dict.items(), key=lambda item: item[1], reverse=True)[:5]

# Metrics on trade-weighted graph
radiality, eccentricity, closeness_centrality = calculate_metrics(G)
betweenness_centrality = calculate_betweenness_centrality(G)

print("\nMetrics on Trade-Weighted Graph:")
print("Top Radialities:", get_top_5(radiality))
print("Top Eccentricities:", get_top_5(eccentricity))
print("Top Closeness Centralities:", get_top_5(closeness_centrality))
print("Top Betweenness Centralities:", get_top_5(betweenness_centrality))

# Metrics on loss-weighted graph
# Create a directed weighted graph using the loss-weighted adjacency matrix
L = create_loss_weight_adj_matrix(G)
G_loss = nx.from_numpy_array(L, create_using=nx.DiGraph)
radiality_loss, eccentricity_loss, closeness_centrality_loss = calculate_metrics(G_loss)
betweenness_centrality_loss = calculate_betweenness_centrality(G_loss)

print("\nMetrics on Loss-Weighted Graph:")
print("Top Radialities:", get_top_5(radiality_loss))
print("Top Eccentricities:", get_top_5(eccentricity_loss))
print("Top Closeness Centralities:", get_top_5(closeness_centrality_loss))
print("Top Betweenness Centralities:", get_top_5(betweenness_centrality_loss))



Metrics on Trade-Weighted Graph:
Top Radialities: [(2, 4.625), (1, 4.5), (0, 3.125), (3, 1.75), (4, 0.0)]
Top Eccentricities: [(4, inf), (0, 7.0), (2, 5.5), (1, 4.5), (3, 1.0)]
Top Closeness Centralities: [(3, 1.0), (2, 0.07407407407407407), (1, 0.07142857142857142), (0, 0.05128205128205128), (4, 0)]
Top Betweenness Centralities: [(2, 0.41666666666666663), (1, 0.25), (3, 0.25), (0, 0.08333333333333333), (4, 0.0)]

Metrics on Loss-Weighted Graph:
Top Radialities: [(2, 3.5), (1, 3.0), (0, 2.5), (3, 1.0), (4, 0.0)]
Top Eccentricities: [(4, inf), (0, 4.0), (1, 3.0), (2, 2.0), (3, 1.0)]
Top Closeness Centralities: [(3, 1.0), (2, 0.16666666666666666), (1, 0.125), (0, 0.1), (4, 0)]
Top Betweenness Centralities: [(2, 0.41666666666666663), (1, 0.25), (3, 0.25), (0, 0.08333333333333333), (4, 0.0)]


### Finding communities

In [ ]:
# Given a digraph G with edge weights, run the louvain community detection algorithm
import community as community_louvain
communities = nx.community.louvain_communities(G, weight='weight')
print("Communities:", communities)

Communities: [{0, 1, 2}, {3, 4}]


### Check the news, if news says few nodes are going to get disrupted, check whether those nodes were bottlenecks in the graph before. 
### If yes, then Issue A Warning, if not we do not care

### Once communites are made, create subgraphs out of these communities, and again run the matrics on these subgraphs

In [ ]:
import networkx as nx

def create_community_subgraphs(G, communities):
    """
    Creates subgraphs for each community, keeping only the edges within the community.
    :param G: Directed weighted graph (NetworkX DiGraph).
    :param communities: List of sets, where each set represents a community of nodes.
    :return: List of subgraphs, one for each community.
    """
    subgraphs = []
    for community in communities:
        # Create subgraph for the current community
        subgraph = G.subgraph(community).copy()  # Use copy to create an independent subgraph
        subgraphs.append(subgraph)
    return subgraphs

# Communities detected in the format: [{0, 1, 2}, {3, 4}]
communities = nx.community.louvain_communities(G, weight='weight')


In [22]:
community_subgraphs = create_community_subgraphs(G, communities)

for i, subgraph in enumerate(community_subgraphs):
    print(f"\n--- Metrics for Community Subgraph {i+1} (Nodes: {list(subgraph.nodes)}) ---")

    # Find metrics for each community subgraph
    radiality, eccentricity, closeness_centrality = calculate_metrics(subgraph)
    betweenness_centrality = calculate_betweenness_centrality(subgraph)
    
    # Function to get top 5 nodes based on metric values
    def get_top_5(metric_dict):
        return sorted(metric_dict.items(), key=lambda item: item[1], reverse=True)[:5]
    
    print("\nTrade Matrix Metrics (Top 5 Nodes):")
    print("Top Radialities:", get_top_5(radiality))
    print("Top Eccentricities:", get_top_5(eccentricity))
    print("Top Closeness Centralities:", get_top_5(closeness_centrality))
    print("Top Betweenness Centralities:", get_top_5(betweenness_centrality))

    # Find metrics on the loss-weighted graph for each community subgraph
    L = create_loss_weight_adj_matrix(subgraph)
    G_loss = nx.from_numpy_array(L, create_using=nx.DiGraph)
    
    radiality_loss, eccentricity_loss, closeness_centrality_loss = calculate_metrics(G_loss)
    betweenness_centrality_loss = calculate_betweenness_centrality(G_loss)
    
    print("\nLoss Matrix Metrics (Top 5 Nodes):")
    print("Top Radialities:", get_top_5(radiality_loss))
    print("Top Eccentricities:", get_top_5(eccentricity_loss))
    print("Top Closeness Centralities:", get_top_5(closeness_centrality_loss))
    print("Top Betweenness Centralities:", get_top_5(betweenness_centrality_loss))



--- Metrics for Community Subgraph 1 (Nodes: [0, 1, 2]) ---

Trade Matrix Metrics (Top 5 Nodes):
Top Radialities: [(1, 3.5), (0, 3.25), (2, 2.25)]
Top Eccentricities: [(2, 5.5), (1, 4.5), (0, 4.0)]
Top Closeness Centralities: [(1, 0.16666666666666666), (0, 0.15384615384615385), (2, 0.11764705882352941)]
Top Betweenness Centralities: [(0, 0.5), (1, 0.5), (2, 0.5)]

Loss Matrix Metrics (Top 5 Nodes):
Top Radialities: [(0, 1.5), (1, 1.5), (2, 1.5)]
Top Eccentricities: [(0, 2.0), (1, 2.0), (2, 2.0)]
Top Closeness Centralities: [(0, 0.3333333333333333), (1, 0.3333333333333333), (2, 0.3333333333333333)]
Top Betweenness Centralities: [(0, 0.5), (1, 0.5), (2, 0.5)]

--- Metrics for Community Subgraph 2 (Nodes: [3, 4]) ---

Trade Matrix Metrics (Top 5 Nodes):
Top Radialities: [(3, 1.0), (4, 0.0)]
Top Eccentricities: [(4, inf), (3, 1.0)]
Top Closeness Centralities: [(3, 1.0), (4, 0)]
Top Betweenness Centralities: [(3, 0.0), (4, 0.0)]

Loss Matrix Metrics (Top 5 Nodes):
Top Radialities: [(0, 1.0